In [2]:
import random
import matplotlib.pyplot as plt
from collections import defaultdict

# --- Classe do Ambiente: Wumpus World ---
class WumpusWorld:
    def __init__(self, size=4):
        self.size = size
        self.agent_pos = (0, 0)
        self.start_pos = (0, 0)

        # Posiciona Wumpus, Ouro e Poços aleatoriamente, mas não na casa inicial
        safe_positions = [(x, y) for x in range(size) for y in range(size) if (x, y) != self.start_pos]
        random.shuffle(safe_positions)

        self.wumpus_pos = safe_positions.pop()
        self.gold_pos = safe_positions.pop()
        self.pits_pos = random.sample(safe_positions, k=size-1) # Adiciona alguns poços

        # Define as recompensas
        self.reward_map = {
            'gold': 1000,
            'wumpus': -1000,
            'pit': -1000,
            'step': -1,
            'win': 1000 # Recompensa por voltar à saída com o ouro
        }
        self.has_gold = False

    def get_percepts(self, pos):
        """ Retorna as percepções do agente na posição atual. """
        x, y = pos
        percepts = {
            'stench': False,
            'breeze': False,
            'glitter': False
        }

        # Verifica Brilho
        if pos == self.gold_pos and not self.has_gold:
            percepts['glitter'] = True

        # Verifica Fedor (adjacente ao Wumpus)
        wx, wy = self.wumpus_pos
        if abs(x - wx) + abs(y - wy) == 1:
            percepts['stench'] = True

        # Verifica Brisa (adjacente a um poço)
        for px, py in self.pits_pos:
            if abs(x - px) + abs(y - py) == 1:
                percepts['breeze'] = True

        return percepts

    def reset(self):
        """ Reseta o ambiente para um novo episódio. """
        self.agent_pos = (0, 0)
        self.has_gold = False
        return self.agent_pos

    def step(self, action):
        """
        Executa uma ação do agente e retorna o novo estado, a recompensa e se o episódio terminou.
        """
        x, y = self.agent_pos

        # Movimenta o agente
        if action == 'up':
            y = min(self.size - 1, y + 1)
        elif action == 'down':
            y = max(0, y - 1)
        elif action == 'left':
            x = max(0, x - 1)
        elif action == 'right':
            x = min(self.size - 1, x + 1)

        self.agent_pos = (x, y)
        new_state = self.agent_pos

        reward = self.reward_map['step']
        done = False

        # Verifica as consequências do movimento
        if self.agent_pos == self.wumpus_pos:
            reward += self.reward_map['wumpus']
            done = True
        elif self.agent_pos in self.pits_pos:
            reward += self.reward_map['pit']
            done = True
        elif self.agent_pos == self.gold_pos and not self.has_gold:
            reward += self.reward_map['gold']
            self.has_gold = True # Agente pegou o ouro

        # Verifica se o agente venceu (voltou à saída com o ouro)
        if self.has_gold and self.agent_pos == self.start_pos:
            reward += self.reward_map['win']
            done = True

        return new_state, reward, done

# --- Classe do Agente com Q-Learning ---
class QLearningAgent:
    def __init__(self, actions, alpha=0.1, gamma=0.9, epsilon=0.9):
        self.actions = actions
        self.alpha = alpha     # Taxa de aprendizado
        self.gamma = gamma     # Fator de desconto
        self.epsilon = epsilon   # Taxa de exploração (vs. explotação)
        self.epsilon_decay = 0.999 # Fator de decaimento do epsilon
        self.epsilon_min = 0.05

        # A Q-table é um dicionário aninhado: q_table[estado][acao] = valor
        self.q_table = defaultdict(lambda: defaultdict(float))

    def choose_action(self, state):
        """ Escolhe uma ação usando a política epsilon-greedy. """
        if random.uniform(0, 1) < self.epsilon:
            # Ação aleatória (Exploração)
            return random.choice(self.actions)
        else:
            # Melhor ação conhecida (Explotação)
            q_values = self.q_table[state]
            if not q_values: # Se não houver valores para este estado, age aleatoriamente
                return random.choice(self.actions)
            return max(q_values, key=q_values.get)

    def update_q_table(self, state, action, reward, next_state):
        """ Atualiza a Q-table com a fórmula do Q-Learning. """
        old_value = self.q_table[state][action]

        # Pega o valor Q máximo para o próximo estado
        next_max_q = 0
        if self.q_table[next_state]:
            next_max_q = max(self.q_table[next_state].values())

        # Fórmula do Q-Learning
        new_value = old_value + self.alpha * (reward + self.gamma * next_max_q - old_value)
        self.q_table[state][action] = new_value

    def update_epsilon(self):
        """ Reduz o epsilon para diminuir a exploração ao longo do tempo. """
        if self.epsilon > self.epsilon_min:
            self.epsilon *= self.epsilon_decay

# --- Loop Principal da Simulação ---
if __name__ == "__main__":
    # Configurações da simulação
    env = WumpusWorld(size=4)
    actions = ['up', 'down', 'left', 'right']
    agent = QLearningAgent(actions=actions, alpha=0.5, gamma=0.9, epsilon=1.0)

    num_episodes = 5000  #numero de iteração do treinamento
    max_steps_per_episode = 1000 #quantidade de movimentos máxima em cada iteração
    all_rewards = []
    wins = 0

    print("Iniciando o treinamento do agente...")
    print(f"Posição do Wumpus: {env.wumpus_pos}")
    print(f"Posição do Ouro: {env.gold_pos}")
    print(f"Posições dos Poços: {env.pits_pos}")
    print("-" * 30)

    # Loop de treinamento por episódios
    for episode in range(num_episodes):
        state = env.reset()
        total_reward = 0
        done = False

        for step in range(max_steps_per_episode):
            action = agent.choose_action(state)
            next_state, reward, done = env.step(action)

            agent.update_q_table(state, action, reward, next_state)

            state = next_state
            total_reward += reward

            if done:
                # Verifica se o episódio terminou com vitória
                if env.has_gold and env.agent_pos == env.start_pos:
                    wins += 1
                break

        all_rewards.append(total_reward)
        agent.update_epsilon() # Decai o epsilon após cada episódio

        if (episode + 1) % 500 == 0:
            print(f"Episódio {episode + 1}/{num_episodes} - Recompensa Média (últimos 100): {sum(all_rewards[-100:])/100:.2f} - Epsilon: {agent.epsilon:.3f}")

    print("\nTreinamento concluído.")
    print(f"Total de vitórias: {wins} de {num_episodes} episódios ({wins/num_episodes*100:.2f}%)")

    # Exibindo a Q-Table aprendida (apenas alguns estados para visualização)
    print("\n--- Amostra da Q-Table Aprendida ---")
    for i in range(env.size):
        for j in range(env.size):
            state = (i, j)
            if agent.q_table[state]:
                best_action = max(agent.q_table[state], key=agent.q_table[state].get)
                print(f"Estado {state}: Melhor ação -> {best_action} (Valores: { {k: round(v, 2) for k,v in agent.q_table[state].items()} })")


Iniciando o treinamento do agente...
Posição do Wumpus: (1, 2)
Posição do Ouro: (1, 0)
Posições dos Poços: [(2, 2), (0, 1), (2, 1)]
------------------------------
Episódio 500/5000 - Recompensa Média (últimos 100): 1166.59 - Epsilon: 0.606
Episódio 1000/5000 - Recompensa Média (últimos 100): 1767.36 - Epsilon: 0.368
Episódio 1500/5000 - Recompensa Média (últimos 100): 1827.71 - Epsilon: 0.223
Episódio 2000/5000 - Recompensa Média (últimos 100): 1967.68 - Epsilon: 0.135
Episódio 2500/5000 - Recompensa Média (últimos 100): 1877.94 - Epsilon: 0.082
Episódio 3000/5000 - Recompensa Média (últimos 100): 1997.88 - Epsilon: 0.050
Episódio 3500/5000 - Recompensa Média (últimos 100): 1967.86 - Epsilon: 0.050
Episódio 4000/5000 - Recompensa Média (últimos 100): 1967.92 - Epsilon: 0.050
Episódio 4500/5000 - Recompensa Média (últimos 100): 1877.97 - Epsilon: 0.050
Episódio 5000/5000 - Recompensa Média (últimos 100): 1937.91 - Epsilon: 0.050

Treinamento concluído.
Total de vitórias: 4540 de 5000 ep